# Welcome to HTMX cookbook

> Please note, this very much WIP. If you have certain recipes that you want to share or want to see, head to [Github](https://github.com/callmephilip/htmx-cookbook)

This cookbook is powered by [FastHTML](https://fastht.ml/). UI is based on [block.css](https://github.com/thesephist/blocks.css)

## Getting started

```
python -m venv .env
source .env/bin/activate
pip install notebook
jupyter notebook
```

The easiest way to run this is to smash `Run -> Run All Cells`. To iterate on a specific recipe, update code in the corresponding cell and rerun it

![Screenshot](https://github.com/callmephilip/htmx-cookbook/blob/main/cookbook.gif?raw=1)

In [ ]:
! pip install --no-cache-dir python-fasthtml git+https://github.com/callmephilip/fasthtml-nb-ext.git

In [2]:
# Setup - you can largely ignore this unless you wan to tweak how the cookbook works

import traceback
from urllib.request import urlopen
from fasthtml.common import *
from fasthtml.jupyter import *
from fasthtml_nb_ext import Playground

# UI
def prettify(f):
    def _f(*args, **kw): return f(*args, **(kw | { "cls": "block " + kw.get("cls","") }))
    return _f

@prettify
def Panel(*content, **kw): return Div(*content, **kw)

@prettify
def Textbox(*content, **kw): return Input(*content, **kw)

@prettify
def Password(*content, **kw): return Input(type="password", *content, **kw)

@prettify
def MultilineTextbox(*content, **kw): return Textarea(*content, **kw)

@prettify
def ActionButton(*content, **kw): return Button(*content, **kw)

def AddressBar(*content, **kw): return Textbox(*content, readonly=True, **kw)

on_response_error = """document.querySelector("#errors").innerHTML = event.detail.xhr.responseText; document.querySelector(".error-box").classList.add("show");"""

def PlaygroundUI(*content, path:str): return ErrorBox(), Panel(AddressBar(value=f"🌎 http://localhost:8000{path}", cls="w-100"), Hr(), *content, HtmxOn("responseError", on_response_error), cls="h-fullish")
def ErrorBox():
    return Panel(cls="error-box")(
        Div(cls="flex flex-row")(
            Div(P("Oh 🦌!")), 
            Div(cls="flex-grow-1 text-align-right")(
                ActionButton(cls="display-inline", onclick="document.querySelector('.error-box').classList.remove('show')")("x")
            )
        ),
        Hr(),
        P(id="errors")
    )


# scripts
htmx,fasthtml_js = "https://unpkg.com/htmx.org@2.0.2", "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js@1.0.4/fasthtml.js"
# styles + fonts
styles,blk_css,font_css = "/styles.css","https://unpkg.com/blocks.css/dist/blocks.min.css","https://fonts.googleapis.com/css2?family=IBM+Plex+Mono:wght@500;700&amp;display=swap"
inline_styles = [Style(urlopen("https://raw.githubusercontent.com/callmephilip/htmx-cookbook/refs/heads/main/styles.css").read().decode('utf-8'))]
hdrs = [Script(src=s) for s in [htmx, fasthtml_js]] + [Link(href=style, rel="stylesheet") for style in [blk_css,styles,font_css]] + inline_styles

def server_error(request: Request, exc: HTTPException): return HTMLResponse(content=''.join(traceback.format_exception(exc)), status_code=500)

Playground.config(default_hdrs=False, hdrs=hdrs, exception_handlers={ 500: server_error, Exception: server_error })

In [ ]:
with Playground(path="/") as playground:
  @playground.rt(path="/")
  def get():
      return PlaygroundUI(
          H1("HTMX cookbook"),
          P("This is very much WIP. Please leave your comments/requests on Github"),
          Div("Let's get started 👇", cls="wrapper animate-bounce block"),
          path="/"
      )


# Basic action

Grab some stuff from a URL on click

In [ ]:
with Playground(path="/get-on-click") as playground:
  @playground.rt("/data")
  def get(): return Span("😱 i am trapped inside the button now!")
  @playground.rt(path="/get-on-click")
  def get(): return PlaygroundUI(ActionButton("Get some data", hx_get="/data"), path="/get-on-click")


# Custom trigger attribute

In [ ]:
from time import sleep


with Playground(path="/html2ft") as playground:
  @dataclass
  class Html2FT: html: str

  @playground.rt("/html2ft/convert")
  def post(data: Html2FT):
      sleep(5)
      return html2ft(data.html)

  on_paste = """
  document.querySelector("#txt-html").addEventListener("paste", function (event) { setTimeout(() => {event.target.blur();}, 500) });
  """

  @playground.rt("/html2ft")
  def get():
    return PlaygroundUI(
        Div(cls="flex flex-row")(
            Div(cls="flex-grow-1 h-300px")(
                P("Convert HTML to FT"),
                MultilineTextbox(name="html", placeholder="paste html here", cls="w-100 h-80", id="txt-html", hx_post="/html2ft/convert", hx_trigger="blur", hx_target="#result", hx_indicator="#processing-indicator"),
                Strong(id="processing-indicator", cls="htmx-indicator")("Processing...")
            ),
            Div(cls="flex-grow-2 h-300px text-align-left", style="padding-left: 5rem;")(
                Code(id="result", cls="white-space-pre-wrap text-align-left")
            )
        ),
        Script(code=on_paste),
        path="/html2ft"
    )

# You can also post some data over

In [ ]:
from password_strength import PasswordStats

with Playground(path="/login") as playground:
  @dataclass
  class LoginData: email: str; pw:str

  @playground.rt("/login")
  def post(ld: LoginData): return Span(f"You are logged in!")

  @playground.rt("/login/validation")
  def post(ld: LoginData):
    strength = int(PasswordStats(ld.pw).strength() * 10) if ld.pw else 0
    if strength > 7:
        c = "green"
    elif strength > 5:
        c = "orange"
    else: c = "red"

    return Div(id="email-validator", hx_swap_oob="true")("✅" if "@" in ld.email else "🔴"), strength > 0 and Div(id="password-validator", hx_swap_oob="true")(
        Div(cls="flex flex-row")(
            *[Div(cls=f"text-{c}")("+") if i + 1 <= strength else Div(cls="tex-gray")("_") for i in range(9)]
        )
    )

  @playground.rt("/login")
  def get():
    return PlaygroundUI(
        Form(hx_post="/login")(
            Div(cls="flex flex-row")(
                Textbox(name="email", placeholder="Email", hx_post="/login/validation", hx_trigger="keyup"),
                Div(id="email-validator")
            ),
            Div(cls="flex flex-row")(
                Password(name="pw", placeholder="Password", hx_post="/login/validation", hx_trigger="keyup"),
                Div(id="password-validator")
            ),
            ActionButton("Go!"),
        ),
        path="/login"
    )
